# Using ML for paramter estimation

In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sl
from scipy import stats
from sklearn import datasets
from sklearn import linear_model
from scipy.optimize import minimize
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


"""Loading the Data"""

dir       = 'data/'
file_name = 'grid1_zheb51fo.xlsx'
UX1       = pd.read_excel(dir+file_name, sheet_name='UX1_Index')
UX2       = pd.read_excel(dir+file_name, sheet_name='UX2_Index')
UX1       = UX1.set_index('Date')
UX2       = UX2.set_index('Date')
UX1.dropna(subset = ["PX_LAST"], inplace=True)   #Getting rid of NaN values
UX2.dropna(subset = ["PX_LAST"], inplace=True)
UX1.sort_index(inplace=True)
UX2.sort_index(inplace=True)
dataset = np.array(UX1.PX_LAST)

In [2]:
"""Specifying the Input & Output (Labels)"""

n=0     #Looking at n previous days to estimate paramteres + today's volatility
X= [[dataset[j] for j in range(i, i+n+1)] for i in range(len(dataset) - n - 1)]
Y= [dataset[i+n+1] for i in range(len(dataset) - n -1)]

# Putting more emphasis on today's data:f

# emphasis = 1    #repearing today's value emphasis times
# for i in range(len(X)):
#     for j in range(emphasis-1):
#         X[i].append(X[i][-1])        
        
# print(X[:10])
# print("\n")
# print(Y[:10])

In [3]:
"""Splitting Data into Train and Test set"""

m_training= 2000
m_test= 1000

X_training= X[:m_training]
Y_training= Y[:m_training]   
X_test= X[m_training:m_training+m_test]
Y_test= Y[m_training:m_training+m_test]

if n==0:
    X_training = np.ravel(X_training)
    X_test = np.ravel(X_test)
    Y_training = np.array(Y_training)
    Y_test = np.array(Y_test)

In [13]:
"""Building the Hypothesis"""

def heston_pde_milstein(V0, k, theta, rho, sigma):
    WT  = np.sqrt( 1 ) * np.random.multivariate_normal(np.array([0, 0]), np.array([[1, rho], [rho, 1]]))[1]
    V1 = np.abs(V0+ k * (theta - V0) * 1 + sigma * np.sqrt(V0) * WT + .25 * sigma**2 * (WT**2 - 1))
    return V1

In [5]:
"""Building the Loss Function"""

#The difference between real label and the predicted one to the power of 2
#l = (heston_pde_milstein(X_training[i], r, k, theta, rho, sigma) - Y_training[i])**2

m=len(X_training)  #Training set size
# k: x[0], theta:x[1], rho:x[2], sigma:x[3]

def Ls(X):
    def heston_inner_func(i):   #calculates the predicted lable for each training sample
        WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]
        V1 =  np.abs(X_training[i] + X[0] * (X[1] - X_training[i]) * 1 + 
                     X[3] * np.sqrt(X_training[i]) * WT + .25 * X[3]**2 * (WT**2 - 1))
        return V1
    Ls = (1/m) * np.sum(np.array([(heston_inner_func(i) - Y_training[i])**2 for i in range(m)]))
    return Ls

In [6]:
"""Re_investigating the Loss Function"""




'Re_investigating the Loss Function'

In [7]:
"""ERM: Empirical Risk Minimization"""

result = minimize(Ls, (1,1,1,1))  #initial values should be given

<ipython-input-5-7e69c1efd057>:11: RuntimeWarning: covariance is not positive-semidefinite.
  WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]
<ipython-input-5-7e69c1efd057>:11: RuntimeWarning: covariance is not positive-semidefinite.
  WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]


In [8]:
best_params = result.x
print("Best Paramteres:", best_params)
print("Minimum Ls:", result.fun)

Best Paramteres: [1.00000079 0.99997342 1.00000365 1.00003907]
Minimum Ls: 258.4466653462006


In [9]:
#Are we trapped inside a loval minima?!!

In [14]:
"""Evaluating the model on test Set"""

k, theta, rho, sigma = best_params[0], best_params[1], best_params[2], best_params[3]
Y_pred = np.array([heston_pde_milstein(X_test[i], k, theta, rho, sigma) for i in range(len(X_test))])
Y_pred = np.ravel(Y_pred)

#True Error
Ld =  (1/len(Y_test)) * np.sum((Y_pred - Y_test)**2)
print(Ld)

360.3667427139515


<ipython-input-13-7abb1314c351>:4: RuntimeWarning: covariance is not positive-semidefinite.
  WT  = np.sqrt( 1 ) * np.random.multivariate_normal(np.array([0, 0]), np.array([[1, rho], [rho, 1]]))[1]


In [ ]:
Y_pred_train = np.array([heston_pde_milstein(X_training[i], k, theta, rho, sigma) for i in range(len(X_training))])
L =  (1/m) * np.sum((Y_pred_train - Y_training)**2)
print(L)

print(Y_pred_train[:50])
print(Y_training[:50])

## Using Neural Network for the Whole task: A Non-Physical Experiment

In [ ]:
#Let's, just for a momemnt, Ignore any pre_knowledge about the subject and see what happens

m_training= 2000
m_test= 1000

X_training=X[:m_training]
Y_training=Y[:m_training]   
X_test=X[m_training:m_training+m_test]
Y_test=Y[m_training:m_training+m_test]

if n==0:
    X_training= np.ravel(X_training)  
    X_test    = np.ravel(X_test)
    
#Let's scale the Volatilities:
M = 100    #M times magnification

X_training= np.array(X_training) * M
Y_training= np.array(Y_training) * M
X_test= np.array(X_test) * M
Y_test= np.array(Y_test) * M

In [ ]:
# NN_R = MLPRegressor(hidden_layer_sizes=(20, 20, 20, 20), max_iter=1000, 
#                     alpha=1e-4, solver='adam', momentum=0.9,
#                     activation='relu', tol=1e-4, learning_rate_init=0.001)

# NN_R.fit(X_training, Y_training)
# Y_pred = NN_R.predict(X_test)
# # print("Predicted Volatilty:\n", Y_pred[100:120])
# # print("\nTrue Volatility:\n",     Y_test[100:120])
# print("\nThe score:", NN_R.score(X_test, Y_test))

In [ ]:
#at this point we don't know if we are getting a good score just beacuse of the similarity between volatlities or
#the model is working properly, we should try to figure out a way to find this!
#let's plot the graphs and see what's going on
plt.style.use('dark_background')
plt.figure(figsize=(13,6))
plt.scatter([t for t in range(len(Y_test))], Y_test, alpha= 0.6,color='gold', marker='.', label='Y_test')
plt.scatter([t for t in range(len(Y_test))], Y_pred, alpha= 0.6,color='b', marker='.', label='Y_pred')
plt.scatter([t for t in range(len(Y_test))], 
            [X_test[i][-1] for i in range(len(Y_test))],alpha= 0.6, color='r', marker='.', label='X_test')
# plt.xlim(550,600)
# plt.ylim(3000,5000)
plt.legend()

In [ ]:
#It seems that the result is merely a translation of today's volalitilies not the predictions for tomorrow!
#but let's measure this preciesly

# Distance_from_tomorrow    = np.sum(np.abs(Y_pred - Y_test))
# Distance_from_today = np.sum(np.abs(
#             np.array(Y_pred - [X_test[i][-1] for i in range(len(list(Y_test)))])))

# plt.figure(figsize=(13,6))
# plt.scatter([t for t in range(len(Y_pred))], Distance_from_today, marker = '+', label = 'Distance to Today')
# plt.scatter([t for t in range(len(Y_pred))], Distance_from_tomorrow, marker = '+', label = 'Distance to Tomorrow')
# # plt.xlim(100,140)
# # plt.ylim(-0.5,1)

# plt.legend()

# print("Distance to Today:\n", np.sum(Distance_from_today))
# print("\nDistance to Tommorow:\n", np.sum(Distance_from_tomorrow))

In [ ]:
#Let's Investigate the behaviour of Distance w.r to n and emphasis(Looking at n previous data)
#Just Run the fist Cell before running this cell

D_today = []  #The list to save all values
D_tomorrow = []

for emphasis in [3]:
    Distance_from_today_list    = []
    Distance_from_tomorrow_list = []
    
    for n in range(0,100,5):
        X= [[dataset[j] for j in range(i, i+n+1)] for i in range(len(dataset) - n - 1)]
        Y= [dataset[i+n+1] for i in range(len(dataset) - n -1)]

        for i in range(len(X)):
            for j in range(emphasis-1):
                X[i].append(X[i][-1])    
                
        m_training = 2000
        m_test     = 1000
        X_training=X[:m_training]
        Y_training=Y[:m_training]   
        X_test=X[m_training:m_training+m_test]
        Y_test=Y[m_training:m_training+m_test]
                 
        M = 10    #M times magnification
        X_training= np.array(X_training) * M
        Y_training= np.array(Y_training) * M
        X_test= np.array(X_test) * M
        Y_test= np.array(Y_test) * M
    
        NN_R = MLPRegressor(hidden_layer_sizes=(20,20,20), max_iter=1000, 
                        alpha=1e-4, solver='adam', momentum=0.9,
                        activation='relu', tol=1e-4, learning_rate_init=0.01)
        NN_R.fit(X_training, Y_training)
        Y_pred = NN_R.predict(X_test)
    
        Distance_from_tomorrow    = np.sum(np.abs(Y_pred - Y_test))
        Distance_from_today = np.sum(np.abs(
            np.array(Y_pred - [X_test[i][-1] for i in range(len(list(Y_test)))])))
        Distance_from_today_list.append(Distance_from_today)
        Distance_from_tomorrow_list.append(Distance_from_tomorrow)

    D_today.append(Distance_from_today_list)
    D_tomorrow.append(Distance_from_tomorrow_list)
    
    #Normalization:
    D_today= np.array(D_today)/(max(np.array(D_today).max(),np.array(D_tomorrow).max()))
    D_tomorrow= np.array(D_tomorrow)/(max(np.array(D_today).max(), np.array(D_tomorrow).max()))
    
    # Plotting the Result of previous part
plt.figure(figsize=(13,7))
# plt.plot([t for t in range(L)], Distance_from_today_list, )
plt.plot([t for t in range(len(D_tomorrow[0]))], D_tomorrow[0],
            marker="o", label = 'tom')
plt.plot([t for t in range(len(D_today[0]))],    D_today[0],
            marker="o", label = 'tod')
# plt.scatter([t for t in range(len(D_tomorrow[0]))], D_tomorrow[1],
#             marker="+",color='b', label = 'Emphasis =3')
# plt.scatter([t for t in range(len(D_tomorrow[0]))], D_tomorrow[2],
#             marker="+",color='r', label = 'Emphasis =5')
# plt.scatter([t for t in range(len(D_tomorrow[0]))], D_tomorrow[3],
#             marker="+",color='0', label = 'Emphasis =10')
plt.xlabel('n')
plt.ylabel('Distance')
plt.legend()
plt.show()

In [ ]:
# Model Selection, Grid Search CV
NN_R = MLPRegressor(max_iter=1000, alpha=1e-4, solver='adam', momentum=0.9,
                    activation='relu', tol=1e-4, learning_rate_init=0.001)
parameters = {'hidden_layer_sizes': [(20),(20,20),(20,20,20),(20,20,20,20),(20,20,20,20,20)]}

gsc = GridSearchCV(NN_R, parameters, cv=5)

gsc.fit(X_training, Y_training)

print("Best parameters set found:")
print(gsc.best_params_)
print("\n")

print("Score with best parameters:")
print(gsc.best_score_)
print("\n")

print("All scores on the grid:")
print(gsc.cv_results_.get('split0_test_score'))
print(gsc.cv_results_.get('split1_test_score'))
print(gsc.cv_results_.get('split2_test_score'))
print(gsc.cv_results_.get('split3_test_score'))
print(gsc.cv_results_.get('split4_test_score'))
print(gsc.cv_results_.get('split5_test_score'))